In [30]:
import copy
from pathlib import Path
import warnings

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters


In [10]:
data_train = pd.read_csv('data/train_preprocessed.csv')
data_train.columns

Index(['id', 'valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10',
       'valeur_PM25', 'is_holiday', 'is_jour_ferie', 'precipitation',
       'wind_speed', 'temperature', 'humidity', 'pressure', 'visibility',
       'global_solar_radiation', 'car_flow', 'Year', 'Month', 'Day',
       'is_weekend', 'DayOfYear', 'HourOfDay', 'Weekday', 'DayOfYear_sin',
       'DayOfYear_cos', 'HourOfDay_sin', 'HourOfDay_cos', 'Weekday_sin',
       'Weekday_cos'],
      dtype='object')

In [11]:
data_train.drop(index=35065, inplace=True)

In [12]:
# Assuming `data` is your DataFrame and `id` is the timestamp column
data_train['id'] = pd.to_datetime(data_train['id'])
data_train = data_train.copy()
data_train['time_idx'] = (data_train['id'] - data_train['id'].min()) / pd.Timedelta(hours=1)
data_train['time_idx'] = data_train['time_idx'].astype(int)
data_train['location']="Montsouris"
data_train['car_flow'] = data_train['car_flow'].fillna(-1)

In [13]:
data_train

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,is_holiday,is_jour_ferie,precipitation,wind_speed,...,HourOfDay,Weekday,DayOfYear_sin,DayOfYear_cos,HourOfDay_sin,HourOfDay_cos,Weekday_sin,Weekday_cos,time_idx,location
0,2020-01-01 00:00:00,42.900000,0.718000,15.7,73.1,64.4,1,0,0.0,1.5,...,0,2,0.017213,0.999852,0.000000,1.000000e+00,0.974928,-0.222521,0,Montsouris
1,2020-01-01 01:00:00,33.600000,0.587000,10.1,74.8,66.0,1,0,0.0,2.6,...,1,2,0.017213,0.999852,0.258819,9.659258e-01,0.974928,-0.222521,1,Montsouris
2,2020-01-01 02:00:00,29.300000,0.400655,5.1,51.0,44.9,1,0,0.0,1.9,...,2,2,0.017213,0.999852,0.500000,8.660254e-01,0.974928,-0.222521,2,Montsouris
3,2020-01-01 03:00:00,30.500000,0.246000,7.2,27.7,25.1,1,0,0.0,1.8,...,3,2,0.017213,0.999852,0.707107,7.071068e-01,0.974928,-0.222521,3,Montsouris
4,2020-01-01 04:00:00,29.300000,0.204000,8.3,15.3,13.6,1,0,0.0,2.2,...,4,2,0.017213,0.999852,0.866025,5.000000e-01,0.974928,-0.222521,4,Montsouris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40987,2024-09-03 18:00:00,10.991566,0.222000,55.1,12.0,5.3,0,0,0.0,1.2,...,18,1,-0.895839,-0.444378,-1.000000,-1.836970e-16,0.781831,0.623490,40986,Montsouris
40988,2024-09-03 19:00:00,14.015630,0.245000,48.2,13.4,7.0,0,0,0.0,1.7,...,19,1,-0.895839,-0.444378,-0.965926,2.588190e-01,0.781831,0.623490,40987,Montsouris
40989,2024-09-03 20:00:00,14.545385,0.234000,44.5,12.4,7.1,0,0,0.0,2.0,...,20,1,-0.895839,-0.444378,-0.866025,5.000000e-01,0.781831,0.623490,40988,Montsouris
40990,2024-09-03 21:00:00,20.489943,0.225000,25.9,10.6,5.4,0,0,0.0,1.4,...,21,1,-0.895839,-0.444378,-0.707107,7.071068e-01,0.781831,0.623490,40989,Montsouris


In [14]:
data_train.columns

Index(['id', 'valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10',
       'valeur_PM25', 'is_holiday', 'is_jour_ferie', 'precipitation',
       'wind_speed', 'temperature', 'humidity', 'pressure', 'visibility',
       'global_solar_radiation', 'car_flow', 'Year', 'Month', 'Day',
       'is_weekend', 'DayOfYear', 'HourOfDay', 'Weekday', 'DayOfYear_sin',
       'DayOfYear_cos', 'HourOfDay_sin', 'HourOfDay_cos', 'Weekday_sin',
       'Weekday_cos', 'time_idx', 'location'],
      dtype='object')

In [20]:
data_train['holiday_type'] = np.where(data_train['is_holiday'] == 1, 'holiday', 
                              np.where(data_train['is_jour_ferie'] == 1, 'ferie', np.nan))
data_train = data_train.drop(columns=['is_holiday','is_jour_ferie'])
data_train['holiday_type']

0        holiday
1        holiday
2        holiday
3        holiday
4        holiday
          ...   
40987        nan
40988        nan
40989        nan
40990        nan
40991        nan
Name: holiday_type, Length: 40991, dtype: object

In [21]:
data_train.columns

Index(['id', 'valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10',
       'valeur_PM25', 'precipitation', 'wind_speed', 'temperature', 'humidity',
       'pressure', 'visibility', 'global_solar_radiation', 'car_flow', 'Year',
       'Month', 'Day', 'is_weekend', 'DayOfYear', 'HourOfDay', 'Weekday',
       'DayOfYear_sin', 'DayOfYear_cos', 'HourOfDay_sin', 'HourOfDay_cos',
       'Weekday_sin', 'Weekday_cos', 'time_idx', 'location', 'holiday_type'],
      dtype='object')

In [24]:
max_prediction_length = 502
max_encoder_length = 365*24*3

In [27]:
training_cutoff = data_train["time_idx"].max() - max_prediction_length
training_cutoff

40488

In [33]:
data_train['Year'] = data_train['Year'].astype(str)
data_train['Month'] = data_train['Month'].astype(str)
data_train['Day'] = data_train['Day'].astype(str)
data_train['is_weekend'] = data_train['is_weekend'].astype(str)
data_train['DayOfYear'] = data_train['DayOfYear'].astype(str)
data_train['HourOfDay'] = data_train['HourOfDay'].astype(str)
data_train['Weekday'] = data_train['Weekday'].astype(str)

In [36]:
training = TimeSeriesDataSet(
    data_train[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target=['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10','valeur_PM25'],
    group_ids=["location"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    time_varying_known_categoricals=['Year','Month', 'Day', 'is_weekend','DayOfYear', 'HourOfDay', 'Weekday','holiday_type'],
    time_varying_known_reals=["time_idx", 'DayOfYear_sin', 'DayOfYear_cos', 'HourOfDay_sin', 'HourOfDay_cos','Weekday_sin', 'Weekday_cos','precipitation', 'wind_speed', 'temperature', 'humidity','pressure', 'visibility', 'global_solar_radiation'],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        'valeur_NO2',
        'valeur_CO',
        'valeur_O3',
        'valeur_PM10',
        'valeur_PM25',
        'car_flow'
    ]
)

In [40]:
validation = TimeSeriesDataSet.from_dataset(training, data_train, predict=True, stop_randomization=True)

In [41]:
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [46]:
import tensorflow as tf
import tensorboard as tb

2024-10-30 18:16:49.554664: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730312209.567741   48782 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730312209.571841   48782 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 18:16:49.589455: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [48]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")

In [49]:
trainer = pl.Trainer(
    max_epochs=50,
    accelerator="cpu",
    enable_model_summary=True,
    gradient_clip_val=0.1,
    limit_train_batches=50,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[early_stop_callback]
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=2,
    dropout=0.1,
    hidden_continuous_size=8,
    loss=QuantileLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    optimizer="Ranger",
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size() / 1e3:.1f}k")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Number of parameters in network: 49.2k


/home/jovyan/.conda/myenv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/jovyan/.conda/myenv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


# Following is not working

In [57]:
from ranger21 import Ranger21


In [60]:
class CustomTFT(TemporalFusionTransformer):
    def configure_optimizers(self):
        # Initialize Ranger21 optimizer
        optimizer = Ranger21(
            self.parameters(),
            lr=0.03,
            num_epochs=50,
            num_batches_per_epoch=len(train_dataloader),
        )
        return optimizer

# Instantiate your custom TFT model
tft = CustomTFT.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=2,
    dropout=0.1,
    hidden_continuous_size=8,
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

/home/jovyan/.conda/myenv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/jovyan/.conda/myenv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
/home/jovyan/.conda/myenv/lib/python3.10/site-packages/pytorch_forecasting/models/temporal_fusion_transformer/__init__.py:143: UserWarning: In pytorch-forecasting models, on versions 1.1.X, the default optimizer defaults to 'adam', if pytorch_optimizer is not installed, otherwise it defaults to 'ranger' from pytorch_optimizer. From version 1.2.0, the default optimizer will be 'adam' regardless of whether pytorch_optimizer is i

Ranger21 optimizer ready with following settings:

Core optimizer = AdamW
Learning rate of 0.03

Important - num_epochs of training = ** 50 epochs **
please confirm this is correct or warmup and warmdown will be off

Warm-up: linear warmup, over 2000 iterations

Lookahead active, merging every 5 steps, with blend factor of 0.5
Norm Loss active, factor = 0.0001
Stable weight decay of 0.0001
Gradient Centralization = On

Adaptive Gradient Clipping = True
	clipping value of 0.01
	steps for clipping = 0.001

Warm-down: Linear warmdown, starting at 72.0%, iteration 11520 of 16000
warm down will decay until 3e-05 lr


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jovyan/.conda/myenv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/jovyan/.conda/myenv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

: 

: 

In [100]:
tft = TemporalFusionTransformer.from_dataset(
    dataset,  # the dataset we defined previously
    learning_rate=0.03,
    hidden_size=16,  # model capacity
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=[1,1,1,1,1],  # Number of target variables
    loss=QuantileLoss()  # Use QuantileLoss for multi-output forecasting
)

In [103]:
trainer = Trainer(
    max_epochs=30,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices="auto" if torch.cuda.is_available() else 1  # Set to 1 for CPU
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [106]:
trainer.fit(tft, train_dataloaders=train_dataloader)


TypeError: `model` must be a `LightningModule` or `torch._dynamo.OptimizedModule`, got `TemporalFusionTransformer`

trainer = Trainer(max_epochs=30, gpus=1 if torch.cuda.is_available() else 0)  # Use 1 GPU if available

In [28]:
data_train["id"] = pd.to_datetime(data_train["id"])
features = ['id', 'is_holiday', 'is_jour_ferie', 'precipitation',
       'wind_speed', 'temperature', 'humidity', 'pressure', 'visibility',
       'global_solar_radiation', 'Year', 'is_weekend', 'DayOfYear',
       'HourOfDay', 'DayOfYear_sin', 'DayOfYear_cos', 'HourOfDay_sin',
       'HourOfDay_cos', 'Weekday_sin', 'Weekday_cos']
data_train["location"] = "Montsouris"
data_train['time_idx'] = ((data_train['id'] - data_train['id'].min()).dt.total_seconds() // 3600).astype("int")
data_train.shape

(40992, 31)

In [29]:
data_train.drop_duplicates()

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,is_holiday,is_jour_ferie,precipitation,wind_speed,...,HourOfDay,Weekday,DayOfYear_sin,DayOfYear_cos,HourOfDay_sin,HourOfDay_cos,Weekday_sin,Weekday_cos,location,time_idx
0,2020-01-01 00:00:00,42.900000,0.718000,15.7,73.1,64.4,1,0,0.0,1.5,...,0,2,0.017213,0.999852,0.000000,1.000000e+00,0.974928,-0.222521,Montsouris,0
1,2020-01-01 01:00:00,33.600000,0.587000,10.1,74.8,66.0,1,0,0.0,2.6,...,1,2,0.017213,0.999852,0.258819,9.659258e-01,0.974928,-0.222521,Montsouris,1
2,2020-01-01 02:00:00,29.300000,0.400655,5.1,51.0,44.9,1,0,0.0,1.9,...,2,2,0.017213,0.999852,0.500000,8.660254e-01,0.974928,-0.222521,Montsouris,2
3,2020-01-01 03:00:00,30.500000,0.246000,7.2,27.7,25.1,1,0,0.0,1.8,...,3,2,0.017213,0.999852,0.707107,7.071068e-01,0.974928,-0.222521,Montsouris,3
4,2020-01-01 04:00:00,29.300000,0.204000,8.3,15.3,13.6,1,0,0.0,2.2,...,4,2,0.017213,0.999852,0.866025,5.000000e-01,0.974928,-0.222521,Montsouris,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40987,2024-09-03 18:00:00,10.991566,0.222000,55.1,12.0,5.3,0,0,0.0,1.2,...,18,1,-0.895839,-0.444378,-1.000000,-1.836970e-16,0.781831,0.623490,Montsouris,40986
40988,2024-09-03 19:00:00,14.015630,0.245000,48.2,13.4,7.0,0,0,0.0,1.7,...,19,1,-0.895839,-0.444378,-0.965926,2.588190e-01,0.781831,0.623490,Montsouris,40987
40989,2024-09-03 20:00:00,14.545385,0.234000,44.5,12.4,7.1,0,0,0.0,2.0,...,20,1,-0.895839,-0.444378,-0.866025,5.000000e-01,0.781831,0.623490,Montsouris,40988
40990,2024-09-03 21:00:00,20.489943,0.225000,25.9,10.6,5.4,0,0,0.0,1.4,...,21,1,-0.895839,-0.444378,-0.707107,7.071068e-01,0.781831,0.623490,Montsouris,40989


In [30]:
max_encoder_length = 7 * 24
max_prediction_length = 502  # Forecast 502 hours into the future

In [31]:
data_train.drop(index=35065, inplace=True)

In [34]:
from pytorch_forecasting import TimeSeriesDataSet

split_idx = int(data_train["time_idx"].max() * 0.8)  # 80% for training, adjust as needed

train_data = data_train[data_train["time_idx"] <= split_idx]
val_data = data_train[data_train["time_idx"] > split_idx]

train_dataset = TimeSeriesDataSet(
    train_data,
    time_idx="time_idx",
    target=["valeur_NO2", "valeur_CO", "valeur_O3", "valeur_PM10", "valeur_PM25"],
    group_ids=["location"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_unknown_reals=["valeur_NO2", "valeur_CO", "valeur_O3", "valeur_PM10", "valeur_PM25"],
    time_varying_known_reals=features,
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

val_dataset = TimeSeriesDataSet(
    val_data,
    time_idx="time_idx",
    target=["valeur_NO2", "valeur_CO", "valeur_O3", "valeur_PM10", "valeur_PM25"],
    group_ids=["location"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_unknown_reals=["valeur_NO2", "valeur_CO", "valeur_O3", "valeur_PM10", "valeur_PM25"],
    time_varying_known_reals=features,
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)


In [51]:
class MultiLoss:
    def __init__(self, losses):
        self.losses = losses

    def __call__(self, y_pred, y_true):
        total_loss = 0
        for i, loss_fn in enumerate(self.losses):
            total_loss += loss_fn(y_pred[..., i], y_true[..., i])
        return total_loss

In [59]:
from pytorch_forecasting.models.temporal_fusion_transformer import TemporalFusionTransformer
from pytorch_lightning import Trainer
from pytorch_forecasting.metrics import QuantileLoss

# Define TFT model
tft = TemporalFusionTransformer.from_dataset(
    train_dataset,
    learning_rate=0.03,  # You may adjust this
    hidden_size=16,  # Size of the network layers
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=[1, 1, 1, 1, 1], 
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

print(f"Number of parameters in model: {tft.size()/1e3:.1f}k")


Number of parameters in model: 54.5k


In [60]:
from pytorch_forecasting.models.temporal_fusion_transformer import TemporalFusionTransformer

In [64]:
import pytorch_lightning as pl
isinstance(tft, pl.LightningModule)

False

In [61]:
trainer = Trainer(
    max_epochs=30,
    gradient_clip_val=0.1,
    callbacks=[EarlyStopping(monitor="val_loss", patience=5)],
    precision=16 if torch.cuda.is_available() else 32,
    log_every_n_steps=10
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [62]:
trainer.fit(
    tft,
    train_dataloaders=train_dataset.to_dataloader(train=True, batch_size=64, num_workers=4),
    val_dataloaders=val_dataset.to_dataloader(train=False, batch_size=64, num_workers=4),
)


TypeError: `model` must be a `LightningModule` or `torch._dynamo.OptimizedModule`, got `TemporalFusionTransformer`

In [65]:
import pytorch_lightning as pl
import torch

class TFTLightningWrapper(pl.LightningModule):
    def __init__(self, tft_model):
        super().__init__()
        self.model = tft_model

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.model.loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.model.loss(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.03)


In [66]:
from pytorch_forecasting.models.temporal_fusion_transformer import TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss

# Instantiate the TemporalFusionTransformer model
tft = TemporalFusionTransformer.from_dataset(
    train_dataset,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=[1, 1, 1, 1, 1],  # Adjusted for each target variable
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

# Wrap the model in the custom LightningModule wrapper
tft_wrapped = TFTLightningWrapper(tft)


In [68]:
trainer = pl.Trainer(
    max_epochs=30
)

trainer.fit(
    tft_wrapped,
    train_dataloaders=train_dataset.to_dataloader(train=True, batch_size=64, num_workers=4),
    val_dataloaders=val_dataset.to_dataloader(train=False, batch_size=64, num_workers=4),
)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type                      | Params | Mode 
------------------------------------------------------------
0 | model | TemporalFusionTransformer | 54.5 K | train
------------------------------------------------------------
54.5 K    Trainable params
0         Non-trainable params
54.5 K    Total params
0.218     Total estimated model params size (MB)
1016      Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jovyan/.conda/myenv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/jovyan/.conda/myenv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/jovyan/.conda/myenv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/jovyan/.conda/myenv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/jovyan/.conda/myenv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/jovyan/.conda/myenv/lib/python3.10

Sanity Checking:   0%|          | 0/2 [00:00<?, ?it/s]

/home/jovyan/.conda/myenv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/jovyan/.conda/myenv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/jovyan/.conda/myenv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/jovyan/.conda/myenv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/jovyan/.conda/myenv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/jovyan/.conda/myenv/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/jovyan/.conda/myenv/lib/python3.10

TypeError: list indices must be integers or slices, not tuple